# Capstone Project - Neighborhood Segmentation & Clustering

In [23]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


### Scraping Data From The Internet

Install BeautifulSoup 4 and lxml if not already installed

In [3]:
pip install BeautifulSoup4

Note: you may need to restart the kernel to use updated packages.


Import BeautifulSoup4 and requests

In [4]:
from bs4 import BeautifulSoup
import requests

Download the info from the website url

In [24]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

Save the info into a variable named "table"

In [25]:
table = pd.read_html(url)

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1045)>

Inspect the contents

In [25]:
table[0:]

[    Postcode           Borough          Neighbourhood
 0        M1A      Not assigned           Not assigned
 1        M2A      Not assigned           Not assigned
 2        M3A        North York              Parkwoods
 3        M4A        North York       Victoria Village
 4        M5A  Downtown Toronto           Harbourfront
 ..       ...               ...                    ...
 283      M8Z         Etobicoke              Mimico NW
 284      M8Z         Etobicoke     The Queensway West
 285      M8Z         Etobicoke  Royal York South West
 286      M8Z         Etobicoke         South of Bloor
 287      M9Z      Not assigned           Not assigned
 
 [288 rows x 3 columns],
                                                   0   \
 0                                                NaN   
 1  NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...   
 2                                                 NL   
 3                                                  A   
 
                          

Save the contents of the first column only to a dataframe, df and print the first five rows

In [26]:
df = table[0]
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


### Wrangle the dataframe to its intended form

Only process cells that have an named Borough. Drop all rows where Borough is "Not assigned"

In [27]:
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace=True)

Print the new dataframe to check

In [28]:
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [10]:
df.isnull() # Check for nans

,Postcode,Borough,Neighbourhood
2,False,False,False
3,False,False,False
4,False,False,False
5,False,False,False
6,False,False,False
...,...,...,...
282,False,False,False
283,False,False,False
284,False,False,False
285,False,False,False


Combine rows with the same postcodes

In [29]:
dfNew = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(list)

dfNew = dfNew.sample(frac=1).reset_index()

dfNew['Neighbourhood']= dfNew['Neighbourhood'].str.join(', ')

In [30]:
dfNew.head()

,Postcode,Borough,Neighbourhood
0,M3L,North York,Downsview West
1,M6P,West Toronto,"High Park, The Junction South"
2,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre"
3,M1H,Scarborough,Cedarbrae
4,M5E,Downtown Toronto,Berczy Park


In [31]:
dfNew.shape[0]

103

Replace each instance of "Not assigned" in Neighbourhood with its corresponding Borough.

In [32]:
dfNew['Neighbourhood'].replace(to_replace ="Not assigned", value=dfNew['Borough'], inplace=True)

In [33]:
dfNew.head()

,Postcode,Borough,Neighbourhood
0,M3L,North York,Downsview West
1,M6P,West Toronto,"High Park, The Junction South"
2,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre"
3,M1H,Scarborough,Cedarbrae
4,M5E,Downtown Toronto,Berczy Park


Check for duplicate rows

In [34]:
dfNew_duplicated = dfNew[dfNew.duplicated()]
print(dfNew_duplicated)

Empty DataFrame
Columns: [Postcode, Borough, Neighbourhood]
Index: []


Check the size of the dataframe

In [35]:
dfNew.shape

(103, 3)

Using the csv file to pull in the coordinates...

In [36]:
geo_coords = pd.read_csv('Geospatial_Coordinates.csv')
# Visualize the data
geo_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Create a dataframe with these coordinates

In [38]:
dfNew['Latitude'] = geo_coords['Latitude'].values
dfNew['Longitude'] = geo_coords['Longitude'].values
dfNew

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3L,North York,Downsview West,43.806686,-79.194353
1,M6P,West Toronto,"High Park, The Junction South",43.784535,-79.160497
2,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.763573,-79.188711
3,M1H,Scarborough,Cedarbrae,43.770992,-79.216917
4,M5E,Downtown Toronto,Berczy Park,43.773136,-79.239476
...,...,...,...,...,...
98,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.706876,-79.518188
99,M5G,Downtown Toronto,Central Bay Street,43.696319,-79.532242
100,M6H,West Toronto,"Dovercourt Village, Dufferin",43.688905,-79.554724
101,M4J,East York,East Toronto,43.739416,-79.588437


Set screen max width for columns

In [39]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [40]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [50]:
 
from geopy.geocoders import Nominatim
print('geopy installed')

geopy installed


In [41]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Check for the number of Buroughs and Neoghbourhoods

In [42]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(dfNew['Borough'].unique()),
        dfNew.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


#### Segmenting via Buroughs that Contain Toronto

In [46]:

toronto_data = dfNew[dfNew['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M6P,West Toronto,"High Park, The Junction South",43.784535,-79.160497
1,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.763573,-79.188711
2,M5E,Downtown Toronto,Berczy Park,43.773136,-79.239476
3,M6J,West Toronto,"Little Portugal, Trinity",43.711112,-79.284577
4,M4E,East Toronto,The Beaches,43.778517,-79.346556
5,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.786947,-79.385975
6,M6S,West Toronto,"Runnymede, Swansea",43.757490,-79.374714
7,M5B,Downtown Toronto,"Ryerson, Garden District",43.770120,-79.408493
8,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.782736,-79.442259
9,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.725900,-79.340923


Getting the geographical coords of Toronto, CA

In [51]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Now, creating a map of Toronto

In [54]:
# create map of Toronto using latitude and longitude values
map_tor = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [55]:
CLIENT_ID = 'O53FBKP4LDDTEE4EORKYZFZQFZQ3BH2IAUROXIXTP3U1LWHP' # your Foursquare ID
CLIENT_SECRET = 'U3WFZMYEPOVH4V4DKIZWXPBTTYXT521ZYQUWQDA4DZ2YEIUZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: O53FBKP4LDDTEE4EORKYZFZQFZQ3BH2IAUROXIXTP3U1LWHP
CLIENT_SECRET:U3WFZMYEPOVH4V4DKIZWXPBTTYXT521ZYQUWQDA4DZ2YEIUZ


Let's explore the first neighborhood in our dataframe.

In [56]:
toronto_data.loc[0,'Neighbourhood']

'High Park, The Junction South'

Get the neighbourhood's lat and long coords

In [59]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are: {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of High Park, The Junction South are: 43.7845351, -79.16049709999999.


Now, let's get the top 100 venues that are in our neighbourhood within a radius of 500 meters.

First, let's create the GET request URL.

In [60]:
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=O53FBKP4LDDTEE4EORKYZFZQFZQ3BH2IAUROXIXTP3U1LWHP&client_secret=U3WFZMYEPOVH4V4DKIZWXPBTTYXT521ZYQUWQDA4DZ2YEIUZ&ll=43.653963,-79.387207&v=20180605&radius=500&limit=100'

Send the GET request and examine the resutls

In [61]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


In [63]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5dcb25009da7ee001c781b4e'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 80,
  'suggestedBounds': {'ne': {'lat': 43.6584630045, 'lng': -79.38099903084075},
   'sw': {'lat': 43.649462995499995, 'lng': -79.39341496915925}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5227bb01498e17bf485e6202',
       'name': 'Downtown Toronto',
       'location': {'lat': 43.65323167517444,
        'lng': -79.38529600606677,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65323167517444,
          

From Foursquare, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from Foursquare.

In [64]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now clean the json and structure it into a pandas dataframe

In [65]:
venues = results['response']['groups'][0]['items']
   
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Japango,Sushi Restaurant,43.655268,-79.385165
2,Cafe Plenty,Café,43.654571,-79.389450
3,Sansotei Ramen 三草亭,Ramen Restaurant,43.655157,-79.386501
4,Rolltation,Japanese Restaurant,43.654918,-79.387424


Let's check how many venues were returned by Foursquare

In [68]:
print("Number of venues returned by Foursquare: ", nearby_venues.shape[0])
#nearby_venues.shape[1]

Number of venues returned by Foursquare:  80


#### Explore Neighborhoods in Toronto

Create a function to repeat the same process to all the neighborhoods in Toronto

In [69]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Using the function on each neighborhood and create a new dataframe called toronto_venues

In [70]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

High Park, The Junction South
Design Exchange, Toronto Dominion Centre
Berczy Park
Little Portugal, Trinity
The Beaches
Forest Hill North, Forest Hill West
Runnymede, Swansea
Ryerson, Garden District
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
The Beaches West, India Bazaar
Davisville
North Toronto West
Parkdale, Roncesvalles
St. James Town
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Moore Park, Summerhill East
The Annex, North Midtown, Yorkville
Rosedale
Commerce Court, Victoria Hotel
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Harbord, University of Toronto
Christie
Roselawn
Brockton, Exhibition Place, Parkdale Village
Harbourfront, Regent Park
The Danforth West, Riverdale
Davisville North
Business Reply Mail Processing Centre 969 Eastern
Lawrence Park
First Canadian Place, Underground city
Church and Wellesley
Studio District
Cabbagetown, St. James Town
Chinatown, Gran

The size of the resulting dataframe?

In [71]:
toronto_venues.shape

(970, 7)

In [72]:
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"High Park, The Junction South",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
1,"Design Exchange, Toronto Dominion Centre",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
2,"Design Exchange, Toronto Dominion Centre",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
3,"Design Exchange, Toronto Dominion Centre",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
4,"Design Exchange, Toronto Dominion Centre",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location


Now, I'll see how many venues were retrurned per neighbourhood

In [74]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",6,6,6,6,6,6
Berczy Park,8,8,8,8,8,8
"Brockton, Exhibition Place, Parkdale Village",94,94,94,94,94,94
Business Reply Mail Processing Centre 969 Eastern,4,4,4,4,4,4
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",100,100,100,100,100,100
"Cabbagetown, St. James Town",9,9,9,9,9,9
Central Bay Street,6,6,6,6,6,6
"Chinatown, Grange Park, Kensington Market",1,1,1,1,1,1
Christie,100,100,100,100,100,100


Let's dig deeper to know how many unique categories of venues there are.

In [75]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 210 uniques categories.


### Some Analysis of Venues in Each Neighborhood

In [79]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Moving Target,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"High Park, The Junction South",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Design Exchange, Toronto Dominion Centre",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Design Exchange, Toronto Dominion Centre",0,0,0

In [80]:
toronto_onehot.shape

(970, 211)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [81]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Moving Target,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.166667,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.166667,0.00,0.000000,0.0000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.166667,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.

Hopefully, we now have a much smaller dataframe size

In [82]:
toronto_grouped.shape

(37, 211)

Print each neighborhood along with the top 5 most common venues

In [86]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----", hood, "----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Adelaide, King, Richmond ----
            venue  freq
0        Pharmacy  0.17
1  Discount Store  0.17
2         Butcher  0.17
3     Coffee Shop  0.17
4   Grocery Store  0.17


---- Berczy Park ----
                  venue  freq
0   Fried Chicken Joint  0.12
1    Athletics & Sports  0.12
2       Thai Restaurant  0.12
3  Caribbean Restaurant  0.12
4                  Bank  0.12


---- Brockton, Exhibition Place, Parkdale Village ----
                   venue  freq
0                   Café  0.07
1                    Bar  0.05
2  Vietnamese Restaurant  0.05
3            Coffee Shop  0.04
4     Chinese Restaurant  0.04


---- Business Reply Mail Processing Centre 969 Eastern ----
                  venue  freq
0              Bus Line  0.25
1     Convenience Store  0.25
2         Grocery Store  0.25
3  Caribbean Restaurant  0.25
4         Moving Target  0.00


---- CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara ----
             

#### Putting thi sinto a dataframe  
but first, sorting the venues in descending order

In [87]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Creating the new dataframe and displaying the top 10 venues for each neighborhood.

In [146]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Pharmacy,Pizza Place,Grocery Store,Butcher,Coffee Shop,Discount Store,Drugstore,Filipino Restaurant,Field,Fast Food Restaurant
1,Berczy Park,Lounge,Bakery,Hakka Restaurant,Caribbean Restaurant,Fried Chicken Joint,Athletics & Sports,Thai Restaurant,Bank,Falafel Restaurant,Farmers Market
2,"Brockton, Exhibition Place, Parkdale Village",Café,Bar,Vietnamese Restaurant,Coffee Shop,Chinese Restaurant,Bakery,Dumpling Restaurant,Vegetarian / Vegan Restaurant,Mexican Restaurant,Donut Shop
3,Business Reply Mail Processing Centre 969 Eastern,Bus Line,Convenience Store,Grocery Store,Caribbean Restaurant,Yoga Studio,Dumpling Restaurant,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Bakery,Cosmetics Shop,Clothing Store,Breakfast Spot,Gastropub


### Cluster Our Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [147]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=3).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
clusters_ = kmeans.labels_
clusters_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 4, 0, 0])

In [148]:
clusters_.astype(int)

array([0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0])

In [150]:
type(clusters_)

numpy.ndarray

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [149]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', clusters_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M6P,West Toronto,"High Park, The Junction South",43.784535,-79.160497,3.0,Bar,Yoga Studio,Drugstore,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market
1,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.763573,-79.188711,0.0,Pizza Place,Breakfast Spot,Medical Center,Mexican Restaurant,Electronics Store,Rental Car Location,Intersection,Yoga Studio,Drugstore,Field
2,M5E,Downtown Toronto,Berczy Park,43.773136,-79.239476,0.0,Lounge,Bakery,Hakka Restaurant,Caribbean Restaurant,Fried Chicken Joint,Athletics & Sports,Thai Restaurant,Bank,Falafel Restaurant,Farmers Market
3,M6J,West Toronto,"Little Portugal, Trinity",43.711112,-79.284577,0.0,Bus Line,Bakery,Park,Bus Station,Fast Food Restaurant,Metro Station,Intersection,Soccer Field,Electronics Store,Fish & Chips Shop
4,M4E,East Toronto,The Beaches,43.778517,-79.346556,0.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Tea Room,Food Court,Japanese Restaurant,Women's Store,Toy / Game Store,Cosmetics Shop,Bakery


Let's visualize this resulting cluster

In [137]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) +' Cluster '+ str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[4],#[cluster -1]),
        fill=True,
        fill_color=rainbow[4],#[cluster -1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Our Clusters

Name clusters according to the most discriminating category / venue

#### Sit Down

In [151]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,0.0,Pizza Place,Breakfast Spot,Medical Center,Mexican Restaurant,Electronics Store,Rental Car Location,Intersection,Yoga Studio,Drugstore,Field
2,Downtown Toronto,0.0,Lounge,Bakery,Hakka Restaurant,Caribbean Restaurant,Fried Chicken Joint,Athletics & Sports,Thai Restaurant,Bank,Falafel Restaurant,Farmers Market
3,West Toronto,0.0,Bus Line,Bakery,Park,Bus Station,Fast Food Restaurant,Metro Station,Intersection,Soccer Field,Electronics Store,Fish & Chips Shop
4,East Toronto,0.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Tea Room,Food Court,Japanese Restaurant,Women's Store,Toy / Game Store,Cosmetics Shop,Bakery
5,Central Toronto,0.0,Chinese Restaurant,Café,Bank,Japanese Restaurant,Yoga Studio,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant
6,West Toronto,0.0,Cafeteria,Yoga Studio,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
7,Downtown Toronto,0.0,Coffee Shop,Ramen Restaurant,Café,Sandwich Place,Restaurant,Sushi Restaurant,Pizza Place,Movie Theater,Lounge,Bubble Tea Shop
8,Downtown Toronto,0.0,Pharmacy,Pizza Place,Grocery Store,Butcher,Coffee Shop,Discount Store,Drugstore,Filipino Restaurant,Field,Fast Food Restaurant
9,Downtown Toronto,0.0,Gym,Coffee Shop,Beer Store,Japanese Restaurant,Bike Shop,Restaurant,Clothing Store,Chinese Restaurant,Italian Restaurant,Concert Hall
10,East Toronto,0.0,Massage Studio,Falafel Restaurant,Coffee Shop,Furniture / Home Store,Bar,Caribbean Restaurant,Miscellaneous Shop,Yoga Studio,Electronics Store,Fish & Chips Shop


#### Country

In [152]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,1.0,Garden,Yoga Studio,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant


#### Airport

In [153]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Downtown Toronto,2.0,Rental Car Location,Drugstore,Yoga Studio,Donut Shop,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant


#### Bars

In [154]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,West Toronto,3.0,Bar,Yoga Studio,Drugstore,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market


#### Sports Venue

In [155]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,Downtown Toronto,4.0,Baseball Field,Drugstore,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
